# Recomendação de filmes utilizando o dataset do MovieLens
## Tratamento dos dados, implementação e comparação entre o método Baseline e o modelo Apriori

### Importar Bibliotecas

In [9]:
!pip install numpy pandas mlxtend wget

In [10]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

### Dados crus do dataset movieLens

In [11]:
import wget
# !python3 -m wget https://github.com/mmanzato/MBABigData/raw/master/ml-20m-compact.tar.gz
# # Botar referência e créditos ao Marcelo Manzato
# !tar -xvzf ml-20m-compact.tar.gz
# # Aprox 400 filmes e 11k usuarios

In [12]:
# Explorar os dados
movies = pd.read_csv('./dataset/movies_sample.csv')
ratings = pd.read_csv('./dataset/ratings_sample.csv')
df = ratings[['userId', 'movieId', 'rating']]
df = df.merge(movies[['movieId', 'title']])
# Mapeamento em idx
map_users = {user: idx for idx, user in enumerate(df.userId.unique())}
map_items = {item: idx for idx, item in enumerate(df.movieId.unique())}
df['userId'] = df['userId'].map(map_users)
df['movieId'] = df['movieId'].map(map_items)

map_title = {}
for _, row in df.iterrows():
    map_title[row.movieId] = row.title

FileNotFoundError: ignored

In [ ]:
movies.head()

In [ ]:
ratings.head()

In [ ]:
df.head()

### Divisão da base em treino e teste

In [ ]:
from sklearn.model_selection import train_test_split
df_train, df_test = train_test_split(df, test_size=.2, random_state=2)

### Funções para obter informações específicas do DataFrame

In [ ]:
# Obter a nota que um usuário deu para um item.
def get_rating(df, userId,movieId):
    if len(df[(df['userId']==userId)&(df['movieId']==movieId)]) == 0:
        return 0
    return (df.loc[(df.userId==userId) & (df.movieId == movieId),'rating'].iloc[0])

get_rating(df, 6102, 413)

In [ ]:
# Obter a lista de todos os filmes que um usuário avaliou.
def get_movie_ids(df, userId):
    if userId not in df['userId'].values:
        return []
    return (df.loc[(df.userId==userId),'movieId'].tolist())

get_movie_ids(df, 0)

In [ ]:
# Obter o título do item dado o seu id.
def get_movie_title(movieId):
    if movieId not in df['movieId'].values:
        return ''
    return (df.loc[(df.movieId == movieId),'title'].iloc[0])

get_movie_title(0)

In [ ]:
# Obter a lista de ratings de um usuário.
def get_user_ratings(df, userId):
    if userId not in df['userId'].values:
        return []
    return (df.loc[(df.userId==userId),'rating'].tolist())

get_user_ratings(df, 0)

In [ ]:
# Obter a média de ratings de um usuário
def get_user_mean(df, userId):
    return np.mean(get_user_ratings(df, userId))

get_user_mean(df, 1)

In [ ]:
# Obter a lista de todos os usuários que avaliaram o filme
def get_user_ids(df, movieId):
    if movieId not in df['movieId'].values:
        return []
    return (df.loc[(df.movieId==movieId),'userId'].tolist())

get_user_ids(df, 10)[:10]

In [ ]:
# Obter todas as notas do filme
def get_movie_ratings(df, movieId):
    if movieId not in df['movieId'].values:
        return []
    return (df.loc[(df.movieId==movieId),'rating'].tolist())

get_movie_ratings(df, 0)[:10]

In [ ]:
# Obter a média de notas do filme
def get_movie_mean(df, movieId):
    return np.mean(get_movie_ratings(df, movieId))

get_movie_mean(df, 0)

In [ ]:
# Obter a lista de ratings de um usuário.
def get_user_movie_rate(df, userId, minRate = 0):
    if userId not in df['userId'].values:
        return []

    if minRate:
        return (df.loc[(df.userId==userId) & \
                (df.rating >= minRate),['movieId', 'rating']])

    return (df.loc[(df.userId==userId),['movieId', 'rating']])

get_user_movie_rate(df, 0, 5)

# Método Baseline
### Método simples para predição de avaliações baseado em tendências de cada usuário e item

> Recomeda filmes considerando o contexto e os dados tanto dos filmes quanto dos usuários, e a associação entre os filmes e os usuários

In [ ]:
# Calcula a média global, o viés referente ao filme e ao usuário
def get_bias(df):
    c = 1
    global_mean = df['rating'].mean()
    movie_list = df['movieId'].unique()
    movie_bias = {}
    for i in movie_list:
        users = get_user_ids(df, i)
        movie_bias[i] = sum((get_rating(df, u, i)-global_mean) for u in users) / (len(users) + c)

    user_list = df['userId'].unique()
    user_bias = {}
    for u in user_list:
        items = get_movie_ids(df, u)
        user_bias[u] = sum((get_rating(df, u, i)-global_mean-movie_bias[i]) for i in items) / (len(items) + c)

    return global_mean, user_bias, movie_bias

In [ ]:
# Recomenda filmes que o usuário ainda não assistiu
def RecommendMovies(df, userId, globalMean, userBias, movieBias, k = 5):
    movie_list = df['movieId'].unique()
    watched = get_movie_ids(df, userId)
    recommend = []
    for i in movie_list:
        if(not i in watched):
             # Calcula a suposta nota para o filme
            recommendation_score = globalMean + userBias[userId] + movieBias[i]

            recommend.append((i, recommendation_score))

      # Ordena de forma crescente a lista de filmes recomendados pela nota
    recommend.sort(key=lambda x: x[1], reverse=True)
    return recommend[:k]

In [ ]:
train_global_mean, train_user_bias, train_movie_bias = get_bias(df_train)

In [ ]:
print(train_user_bias)
print(train_movie_bias)

### TODO: Avaliação do desempenho do algoritmo baseline

### Método que para cada usuário pega no conjunto de teste os filmes que o usuário avaliou bem (rating > 3 por exemplo) e verifica se este filme foi de fato recomendado pelo método Baseline na função RecommendMovies
### Usar plots?

In [ ]:
# Verifica se a recomendação sugerida pelo método Baseline é um filme
# que o usuário realmente assistiu e deu uma nota maior ou igual a minRate.
def check_recomendations(dfTrain, dfTest, globalMean, userBias,
                         movieBias, users, minRate, k):

    # Array com a quantidade de vídeos assistidos & recomendados
    # corretamente pelo baseline para cada usuário
    found = [0]*len(users)

    for idx, user in enumerate(users):
        recomendation = RecommendMovies(dfTrain, user, globalMean,
                                        userBias, movieBias, k=k)

        test_movies = get_user_movie_rate(dfTest, user, minRate=minRate)

        test_movies_list = []
        if len(test_movies) != 0:
            test_movies_list = test_movies['movieId'].tolist()

            for movie in recomendation:
                # Verifica se o filme (idMovie) recomendado está na lista de filmes assistidos
                if movie[0] in test_movies_list:
                    found[idx] += 1

    return found

In [ ]:
users = df_test['userId'].unique().tolist()
min_rate = 3
found_5 = check_recomendations(df_train, df_test, train_global_mean, train_user_bias,
                             train_movie_bias, users, min_rate, 5)
found_10 = check_recomendations(df_train, df_test, train_global_mean, train_user_bias,
                             train_movie_bias, users, min_rate, 10)
found_20 = check_recomendations(df_train, df_test, train_global_mean, train_user_bias,
                             train_movie_bias, users, min_rate, 20)
found_30 = check_recomendations(df_train, df_test, train_global_mean, train_user_bias,
                             train_movie_bias, users, min_rate, 30)
found_40 = check_recomendations(df_train, df_test, train_global_mean, train_user_bias,
                             train_movie_bias, users, min_rate, 40)

print('Média de recomendações vistas para', 5,'recomendações do baseline:', np.mean(found_5))
print('Média de recomendações vistas para', 10,'recomendações do baseline:', np.mean(found_10))
print('Média de recomendações vistas para', 20,'recomendações do baseline:', np.mean(found_20))
print('Média de recomendações vistas para', 30,'recomendações do baseline:', np.mean(found_30))
print('Média de recomendações vistas para', 40,'recomendações do baseline:', np.mean(found_40))

In [ ]:
# Erro médio absoluto entre as notas previstas e as notas de avaliações reais
amt_movies = df_test['movieId'].unique().size

users = df_test['userId'].unique().tolist()
users.sort()
amt_users = len(users)

# Array que armazenará a diferença absoluta média para cada usuário
diff = []
for user in users:
    # Nota de recomendação para todos os filmes dado o usuário i
    recomendation = RecommendMovies(df_train, user, train_global_mean,
                                        train_user_bias, train_movie_bias, k=amt_movies)
    watched_movies = get_user_movie_rate(df_test, user)

    mean = []
    if len(watched_movies) > 0:
        idmovies = watched_movies['movieId'].tolist()
        ratemovies = watched_movies['rating'].tolist()

        dictMovieRate = {idMovie:rateMovie for idMovie, rateMovie in zip(idmovies, ratemovies)}

        for idx, movie in enumerate(recomendation):
            # Se o usuário já assistiu o filme, calcula-se a diferença absoluta e adiciona-se ao array
            if dictMovieRate.get(movie[0], None):
                mean.append(abs(dictMovieRate[movie[0]] - movie[1]))

        if mean:
            diff.append((user, np.mean(mean)))
            mean = []

print(diff)

In [ ]:
# Geração de vetor apenas com os erros para plot
diff_errors = [erro[1] for erro in diff]

# Média global dos erros
mean_error = np.mean(diff_errors)

In [ ]:
plt.figure(figsize=(15, 6))
plt.bar(range(len(diff_errors)), diff_errors, color=np.where(diff_errors < mean_error, 'red', 'blue'))

plt.axhline(y=np.mean(diff_errors), color='r', linestyle='--', label='Média')

plt.xlabel('Usuário')
plt.ylabel('Erro médio')

upper = np.sum(diff_errors >= mean_error)
bottom = len(diff_errors) - upper
legend = f'Valores acima da média: {upper}\
            \nValores abaixo da média: {bottom}'
plt.legend(title=legend)

plt.title('Erro médio das avaliações para cada usuário')
plt.show()

# Modelo Apriori

> Recomenda filmes considerando principalmente o contexto dos filmes e a relação (associação) entre eles

### Pré-processamento e criação da tabela de filmes assistidos

In [ ]:
df_pivot = df.pivot(index='userId', columns='title', values='rating').fillna(0)

In [ ]:
df_pivot = df_pivot.astype('int64')

In [ ]:
df_pivot = df_pivot.applymap(lambda x: 1 if x > 0 else 0)

In [ ]:
df_pivot.head()


### Treinando o modelo

In [ ]:
from mlxtend.frequent_patterns import apriori

frequent_itemset = apriori(df_pivot, min_support=0.07, use_colnames=True)

In [ ]:
frequent_itemset.head()


In [ ]:
from mlxtend.frequent_patterns import association_rules

rules = association_rules(frequent_itemset, metric="lift", min_threshold=1)


In [ ]:
rules.head()

### Resultados


In [ ]:
df_res = rules.sort_values(by=['lift'], ascending=False)
df_res.head()


### Testando o modelo

In [ ]:
#movie_test = 'I, Robot (2004)
movie_list_test = get_movie_ids(df, 1) # Pega a lista de ids de filmes que o usuário 1 avaliou

# Pega os respectivos títulos
for i in range(len(movie_list_test)):
	movie_list_test[i] = get_movie_title(movie_list_test[i])

movie_list_test

In [ ]:
# Com base nos filmes avaliados, pega as recomendações de cada um e armazena
movies = []
for movie in movie_list_test:
    df_test = df_res[df_res['antecedents'].apply(lambda x: len(x) == 1 and next(iter(x)) == movie)]
    df_test = df_test[df_test['lift'] > 1.5]
    movies.extend(df_test['consequents'].values)

In [ ]:
movie_list = []
for movie in movies:
    for title in movie:
        movie_list.append(title)

In [ ]:
from collections import Counter

# Ordena o vetor pelos mais mais repetidos primeiro
def sort_by_frequency(array):
    # Contar a frequência de cada elemento no array
    count = Counter(array)

    # Ordenar o array com base na contagem de repetições de cada elemento
    sorted_array = sorted(array, key=lambda x: count[x], reverse=True)

    return sorted_array

# Remove os repetidos depois da ordenação
def remove_repeated(array):
    new_array = []
    seen_items = set()

    for movie in array:
        if movie not in seen_items:
            new_array.append(movie)
            seen_items.add(movie)

    return new_array


In [ ]:
movie_list = remove_repeated(sort_by_frequency(movie_list))
movie_list[:10]

### Avaliação do modelo Apriori

Este teste gera todas regras geradas, para averiguar a preditibilidade das regras que o algoritmo gera.

In [ ]:
allRules = association_rules(frequent_itemset, metric="lift")
evaluate = allRules.sort_values(by=['lift'], ascending=True)

import matplotlib.pyplot as plt
plt.figure(figsize=(10, 6))
plt.bar(range(len(evaluate)), evaluate['lift'], align='center', color='skyblue')
plt.axhline(y=1, color='red', linestyle='--', linewidth=2, label='Limiar de Lift Preditivo')
plt.xlabel('Regras de Associação')
plt.ylabel('Valor de Lift')
plt.title('Lift das Regras de Associação')
plt.legend()
plt.show()

# TODO
# Modelo K-Nearest-Neighbors

> Recomenda filmes com base nas preferências de usuários semelhantes

In [ ]:
!pip install scikit-surprise

from surprise import Dataset, Reader, KNNBasic

reader = Reader(rating_scale=(1, 5))
data = Dataset.load_from_df(df[['userId', 'movieId', 'rating']], reader)

In [ ]:
# Essa função Percorre todos os dados de treinamento disponíveis e extrai informações necessárias para construir o conjunto
# de treinamento. Isso inclui o conjunto completo de usuários, itens e avaliações.
trainset = data.build_full_trainset()

# Aqui estabelecemos as opções de similaridade do KNN usando cosseno (por conta de sua invariância a escala) e explicitando
# que as recomendações são baseadas nos usuários
sim_options = {'name': 'cosine', 'user_based': True}
#model = KNNBasic(sim_options=sim_options)
#model.fit(trainset)

In [ ]:
user_id_to_predict = 1  # Aqui é estabelecido o usuário em questão
items_to_ignore = df[df['userId'] == user_id_to_predict]['movieId'].tolist() # Aqui guardamos os filmes já avaliados pelo usuário

# Obtém IDs de filmes ainda não avaliados pelo usuário
all_movie_ids = df['movieId'].unique()
movies_to_predict = [movie_id for movie_id in all_movie_ids if movie_id not in items_to_ignore]

# Gera previsões para os filmes não avaliados
predictions = [model.predict(user_id_to_predict, movie_id) for movie_id in movies_to_predict]

# Organiza as previsões em ordem decrescente de estimativa de classificação
predictions.sort(key=lambda x: x.est, reverse=True)

# Obtém os IDs dos filmes recomendados
recommended_movie_ids = [prediction.iid for prediction in predictions]

# Mapeia os IDs dos filmes recomendados para os títulos reais.
recommended_movies = df[df['movieId'].isin(recommended_movie_ids)][['movieId', 'title']].drop_duplicates()

recommended_movies[:10]